In [124]:
import re
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
import math
import random
import sys
import matplotlib.pyplot as plt
from keras.utils import plot_model
import pandas as pd
import ezodf

In [125]:


def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
x =  read_ods(filename = "sentenceGenerator.ods")
sentences = x["act"]+" "+ x["sentence"]
print(x)
modSentence = ""
for eachSentence in sentences:
    modSentence += eachSentence + " "
print(modSentence)

       act                                           sentence slots
0    greet                                         Hey there!  None
1    greet                                                 Hi  None
2    greet                             Hey how may I Help you  None
3    greet                                              Hello  None
4      bye                                           Good bye  None
5      bye                     Thank you. Take care good bye.  None
6  request                    can I have your request number.  None
7  request     Can you please provide me your request number.  None
8  request  I need your request number to go ahead with yo...  None
greet Hey there! greet Hi greet Hey how may I Help you greet Hello bye Good bye bye Thank you. Take care good bye. request can I have your request number. request Can you please provide me your request number. request I need your request number to go ahead with your request 


### Load File and Build Vocabulary

In [126]:
data_path = "test_data_long.txt"
#raw_text = open(data_path).read().lower()
raw_text = modSentence.lower()
import re
# generate list of unique characters, but only include words and some punctuation marks
pattern = re.compile('[a-z]+|\!|\n|\.|,|;')
all_words = re.findall(pattern, raw_text)
print(all_words)

unique_words = sorted(set(all_words))

word_to_int = dict((c, i) for i, c in enumerate(unique_words))

# print(unique_words)
# print(word_to_int)

# later used to make outputs more readable by converting ints back to characters
int_to_word = dict((i, c) for i, c in enumerate(unique_words))

total_num_words = len(all_words)
len_vocab = len(unique_words)

print("Total number of words:\t" + str(total_num_words))
print("Length of vocabulary:\t" + str(len_vocab))

['greet', 'hey', 'there', '!', 'greet', 'hi', 'greet', 'hey', 'how', 'may', 'i', 'help', 'you', 'greet', 'hello', 'bye', 'good', 'bye', 'bye', 'thank', 'you', '.', 'take', 'care', 'good', 'bye', '.', 'request', 'can', 'i', 'have', 'your', 'request', 'number', '.', 'request', 'can', 'you', 'please', 'provide', 'me', 'your', 'request', 'number', '.', 'request', 'i', 'need', 'your', 'request', 'number', 'to', 'go', 'ahead', 'with', 'your', 'request']
Total number of words:	57
Length of vocabulary:	30


### Create Training Data from Text File

Total patterns:	55


In [128]:
#MODIFYING PREPROCESSING DATA
sequence_length = 1 # max number of words to consider at a time.
                    # this means that each trainig set (training pattern) will be comprised of 20 time steps
step_window = 1

# set up x and y
# convert the words into integers
x_data = [] # list of lists
y_data = []
sents = []
for each in sentences:
    sents.append(each.lower())
for j in range(len(sents)):
    pattern = re.compile('[a-z]+|\!|\n|\.|,|;')
    all_words = re.findall(pattern, sents[j])
    print(all_words)
    for i in range(0, len(all_words) - sequence_length, step_window):

        # extract the first n words (length sequence_length): our "x"
        sequence_in = all_words[i : i+sequence_length]
        print("in", sequence_in)

        # extract last word for this window: our "y" (target)
        word_out = all_words[i+sequence_length]
        print("out", word_out)

        # print('\nx: ' + str(sequence_in) + '\n' + 'y: ' + word_out)
        # print()

        # store corresponding integer for each character in the input sequence
        x_data.append(sequence_in)
        y_data.append(word_out)

num_train_patters = len(x_data)
print('Total patterns:\t' + str(num_train_patters))

['greet', 'hey', 'there', '!']
in ['greet']
out hey
in ['hey']
out there
in ['there']
out !
['greet', 'hi']
in ['greet']
out hi
['greet', 'hey', 'how', 'may', 'i', 'help', 'you']
in ['greet']
out hey
in ['hey']
out how
in ['how']
out may
in ['may']
out i
in ['i']
out help
in ['help']
out you
['greet', 'hello']
in ['greet']
out hello
['bye', 'good', 'bye']
in ['bye']
out good
in ['good']
out bye
['bye', 'thank', 'you', '.', 'take', 'care', 'good', 'bye', '.']
in ['bye']
out thank
in ['thank']
out you
in ['you']
out .
in ['.']
out take
in ['take']
out care
in ['care']
out good
in ['good']
out bye
in ['bye']
out .
['request', 'can', 'i', 'have', 'your', 'request', 'number', '.']
in ['request']
out can
in ['can']
out i
in ['i']
out have
in ['have']
out your
in ['your']
out request
in ['request']
out number
in ['number']
out .
['request', 'can', 'you', 'please', 'provide', 'me', 'your', 'request', 'number', '.']
in ['request']
out can
in ['can']
out you
in ['you']
out please
in ['please']
o

### Prepare Training and Testing Data

In [129]:
x = np.zeros((num_train_patters, sequence_length, len_vocab))
y = np.zeros((num_train_patters, len_vocab))

# encode all data into one-hot vectors
for i, sentence in enumerate(x_data):
    for t, word in enumerate(sentence):
        x[i, t, word_to_int[word]] = 1
    y[i, word_to_int[y_data[i]]] = 1
    
print(y[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]


### Define Model

In [130]:
learning_rate = 0.01
optimizer = RMSprop(lr=learning_rate)
num_memory_units = 256

model = Sequential()

# model.add(LSTM(num_memory_units, return_sequences=True, input_shape=(sequence_length, len_vocab)))
model.add(LSTM(num_memory_units, input_shape=(sequence_length, len_vocab)))
model.add(Dropout(0.2))
# model.add(LSTM(num_memory_units))
# model.add(Dropout(0.2))
model.add(Dense(len_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [131]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 256)               293888    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 30)                7710      
_________________________________________________________________
activation_5 (Activation)    (None, 30)                0         
Total params: 301,598
Trainable params: 301,598
Non-trainable params: 0
_________________________________________________________________


### Train Model

In [132]:
def add_temperature(predictions, temperature=1.0):

    predictions = np.asarray(predictions).astype('float64')
    predictions = np.log(predictions) / temperature
    exp_predictions = np.exp(predictions)
    predictions = exp_predictions / np.sum(exp_predictions)
    
    probabilities = np.random.multinomial(1, predictions, 1)
    return np.argmax(probabilities)

In [133]:
num_iterations = 50
batch_size = 128
words_to_generate = 300

prev_loss = math.inf
loss_history = []
accuracy_history = []

val_loss_history = []
val_accuracy_history = []

# train the model, output generated text after each iteration
for i in range(num_iterations):
    
    print('\n' + '-'*10 + ' epoch ' + str(i+1) + '/' + str(num_iterations) + ' ' + '-'*10)
        
    history = model.fit(x, y, batch_size=batch_size, epochs=1)
    
    curr_loss = history.history['loss'][0]
    loss_history.append(curr_loss)
    
    # save weights if loss improves
    if (curr_loss < prev_loss):
        print("Loss improved from " + str(prev_loss) + " to " + str(curr_loss) + ". Saving weights.")
        model.save_weights('weights_epoch-{}_loss-{}.hdf5'.format(i, curr_loss))
        prev_loss = curr_loss
    
    sentence_index = random.randint(0, len(sents)- 1)
    # start_index = 0
    pattern = re.compile('[a-z]+|\!|\n|\.|,|;')
    all_words = re.findall(pattern, sents[sentence_index])
    print(all_words)
    start_index = random.randint(0, len(all_words) - sequence_length - 1)
    seed_sentence = all_words[start_index : start_index + sequence_length]

    print('\n-> seed: "' + ' '.join(seed_sentence) + '" ...\n')

    for i in range(words_to_generate):
        
        x_input = np.zeros((1, sequence_length, len_vocab))
        for t, word in enumerate(seed_sentence):
            x_input[0, t, word_to_int[word]] = 1.

        predictions = model.predict(x_input, verbose=0)[0]
        
        if i == num_iterations-1:
            final_predicted = predictions
        
        # predicted_word_index = add_temperature(predictions, 0.5)
        predicted_word_index = np.argmax(predictions)
        predicted_word = int_to_word[predicted_word_index]

        seed_sentence = seed_sentence[1:] + list([predicted_word])

        if re.match('[a-z]', predicted_word):
            sys.stdout.write(" " + predicted_word)
        else:
            sys.stdout.write(predicted_word)
    
        sys.stdout.flush()
        
    print()


---------- epoch 1/50 ----------
Epoch 1/1
48/48 [==============================] - 2s 31ms/step - loss: 3.3955
Loss improved from inf to 3.395474672317505. Saving weights.
['greet', 'hi']

-> seed: "greet" ...

 hey request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your request number. your reque

 hey there! your request number. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take ca

 number. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care g

 with your request number. take care good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye 

! your request number. take care good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good

...

 with your request number. take care good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye 

. take care good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good bye good by

 hey how may i help you. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good bye. take care good 

 hey there!. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank yo

 how may i have your request number. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. take care good bye thank you. tak

In [134]:
loss = model.evaluate(x, y, batch_size=batch_size, verbose=1)
print("loss: ", loss)

48/48 [==============================] - 0s 6ms/step
loss:  0.48440441489219666


In [135]:
print('loss history:')
print(loss_history)

# plt.figure(figsize=(15,8))
# plt.rc('font', size=20)
# plt.plot(loss_history, lw=3, c='orange')
# plt.title('Cross Entropy Loss of LSTM Model over Epoch Iterations', fontsize=25)
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.savefig("loss.png")
# plt.grid()
# plt.show()

loss history:
[3.395474672317505, 3.198737859725952, 2.9761435985565186, 2.711524724960327, 2.437443971633911, 2.145151138305664, 1.8930453062057495, 1.6366459131240845, 1.4104185104370117, 1.2061171531677246, 1.0306482315063477, 0.9120805859565735, 0.8178045153617859, 0.7296611666679382, 0.6633129715919495, 0.6413480639457703, 0.610297441482544, 0.5921980738639832, 0.5637121796607971, 0.5576668381690979, 0.5370296835899353, 0.5312871336936951, 0.5279210805892944, 0.5233283042907715, 0.5134761333465576, 0.5312231779098511, 0.5265161395072937, 0.4866659641265869, 0.5106019377708435, 0.49021396040916443, 0.4983659088611603, 0.5080954432487488, 0.5029013156890869, 0.491021066904068, 0.49744272232055664, 0.4985480308532715, 0.509011447429657, 0.49676480889320374, 0.5027754902839661, 0.47752511501312256, 0.47074711322784424, 0.49582505226135254, 0.4957471787929535, 0.5065919756889343, 0.4780445396900177, 0.49594974517822266, 0.5019839406013489, 0.4958372116088867, 0.4791860282421112, 0.5059

In [120]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 256)               276480    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 13)                3341      
_________________________________________________________________
activation_4 (Activation)    (None, 13)                0         
Total params: 279,821
Trainable params: 279,821
Non-trainable params: 0
_________________________________________________________________
None


In [121]:
plot_model(model, to_file='model_plot.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.